In [ ]:
# Torch-related imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset

# Scikit-learn-related imports
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score

# Nibabel and Scipy imports (for handling fMRI and image processing)
import nibabel as nib
import scipy.ndimage as ndimage  # For smoothing

# NumPy, Matplotlib, and Seaborn (for data manipulation and visualization)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# OS for file system operations
import os

In [ ]:
import torch
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Preprocessing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = os.path.join('/content/drive', 'My Drive', 'UCR', '2-2024', 'InvCC', 'ADHD200', 'Datasets', 'preprocessed')

import os
from sklearn.utils.class_weight import compute_class_weight

root_dir = os.path.join('data', 'preprocessed')

tdc_dir = os.path.join(root_dir, 'TDC')
adhd_dir = os.path.join(root_dir, 'ADHD')

# To save autoencoder state dict
save_path = os.path.join(root_dir, 'autoencoder.pt')

# Recursively find all .nii.gz files in TDC and ADHD folders
tdc_file_paths = [
    os.path.join(root, file)
    for root, _, files in os.walk(tdc_dir)
    for file in files if file.endswith('.nii.gz')
]

adhd_file_paths = [
    os.path.join(root, file)
    for root, _, files in os.walk(adhd_dir)
    for file in files if file.endswith('.nii.gz')
]

# Assuming tdc_file_paths and adhd_file_paths were correctly populated
tdc_labels = [0] * len(tdc_file_paths)  # Create labels for TDC
adhd_labels = [1] * len(adhd_file_paths)  # Create labels for ADHD

# Combine file paths and labels
file_paths = tdc_file_paths + adhd_file_paths
labels = tdc_labels + adhd_labels

# Verify lengths
print(f"Total file paths: {len(file_paths)}")  # Should be 5100
print(f"Total labels: {len(labels)}")  # Should also be 5100

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

print(f'Class weights: ', class_weights)

In [ ]:
import torch
import nibabel as nib
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np
from scipy import ndimage

class FMRI_Dataset(Dataset):
    def __init__(self, file_paths, labels, max_shape, smoothing_sigma=1, augment=False):
        self.file_paths = file_paths  # List of paths to the fMRI data files
        self.labels = labels  # Corresponding labels
        self.max_shape = max_shape  # Shape to pad all inputs to (e.g., [1, 61, 73, 61])
        self.smoothing_sigma = smoothing_sigma  # Standard deviation for Gaussian smoothing
        self.augment = augment  # Apply augmentations if True

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # Load fMRI data
        fmri_img = nib.load(self.file_paths[idx])
        data = fmri_img.get_fdata()

        # Apply Gaussian smoothing
        data = self.smooth_data(data)

        # Apply augmentations if enabled
        if self.augment:
            data = self.apply_augmentations(data)

        # Normalize the data
        data = self.normalize_data(data)

        # Convert to tensor and add missing dimensions
        data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0)

        # Pad the tensor to the specified max_shape
        data_padded = F.pad(data_tensor, pad=self.calculate_padding(data_tensor.shape), mode='constant', value=0)

        # Ensure the final shape matches max_shape
        data_padded = data_padded.view(*self.max_shape)

        # Get the label
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return data_padded, label

    def apply_augmentations(self, data):
        data = self.add_noise(data)
        data = self.random_rotate(data)
        data = self.random_intensity_shift(data)
        return data

    def add_noise(self, data, mean=0, std=0.01):
        noise = np.random.normal(mean, std, data.shape)
        return data + noise

    def random_rotate(self, data):
        angles = np.random.uniform(-5, 5, size=3)
        return ndimage.rotate(data, angle=angles[0], axes=(1, 2), reshape=False, mode='nearest')

    def random_intensity_shift(self, data, shift_limit=0.05):
        shift_value = np.random.uniform(-shift_limit, shift_limit)
        return data + shift_value

    def calculate_padding(self, current_shape):
        padding = []
        for current_dim, max_dim in zip(reversed(current_shape), reversed(self.max_shape)):
            pad_total = max_dim - current_dim
            padding.append(pad_total // 2)
            padding.append(pad_total - (pad_total // 2))
        return padding

    def normalize_data(self, data):
        mean = data.mean()
        std = data.std()
        return (data - mean) / std if std > 0 else data

    def smooth_data(self, data):
        return ndimage.gaussian_filter(data, sigma=self.smoothing_sigma)

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedShuffleSplit

# Assuming you already have file_paths and labels defined as in your previous code

# Parameters
batch_size = 4
num_classes = 2
max_shape = [1, 61, 73, 61]

# Stratified Shuffle Split
labels = np.array(labels)
dataset = FMRI_Dataset(file_paths, labels, max_shape)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)  # 70% train, 30% test

for train_index, test_index in sss.split(file_paths, labels):
    train_file_paths, test_file_paths = np.array(file_paths)[train_index], np.array(file_paths)[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]

# Further split the test set into validation and test sets
sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)  # 50% of the test set for validation
for val_index, test_index in sss_val.split(test_file_paths, test_labels):
    val_file_paths, final_test_file_paths = np.array(test_file_paths)[val_index], np.array(test_file_paths)[test_index]
    val_labels, final_test_labels = test_labels[val_index], test_labels[test_index]

# Print the results
print(f"Training set size: {len(train_file_paths)}")
print(f"Validation set size: {len(val_file_paths)}")
print(f"Test set size: {len(final_test_file_paths)}")

# Create datasets
train_dataset = FMRI_Dataset(train_file_paths.tolist(), train_labels.tolist(), max_shape)
val_dataset = FMRI_Dataset(val_file_paths.tolist(), val_labels.tolist(), max_shape)
test_dataset = FMRI_Dataset(final_test_file_paths.tolist(), final_test_labels.tolist(), max_shape)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import random

# Sample based on actual length
sample_size = min(4, len(file_paths))  # Adjust sample size to available data
sample_indices = random.sample(range(len(file_paths)), sample_size)

for idx in sample_indices:
    print(f"File Path: {file_paths[idx]}")
    print(f"Label: {labels[idx]}")
    data = nib.load(file_paths[idx]).get_fdata()
    print(f"Data Shape: {data.shape}\n")


# CNN-AE

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class CNN_Autoencoder(nn.Module):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.Conv3d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.Conv3d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=(1, 1, 1)),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.ConvTranspose3d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=(1, 0, 1)),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=(0, 0, 0)),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 1, kernel_size=3, stride=2, padding=1, output_padding=(0, 0, 0)),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = CNN_Autoencoder().to(device)

# Generate random input
inputs = torch.rand((4, 1, 61, 73, 61)).to(device)  # Example input shape
output = autoencoder(inputs)
print(output.shape)  # Should match the input shape [4, 1, 61, 73, 61]

# Loss and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for autoencoder
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
# Number of epochs
epochs = 100

# Track the best validation loss
best_val_loss = float('inf')

for epoch in range(epochs):
    # Training phase
    autoencoder.train()  # Set the model to training mode
    total_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)  # Send to GPU if available

        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)  # Compare reconstruction with original input

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {avg_train_loss}")

    # Validation phase
    autoencoder.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation
        for val_inputs, val_labels in val_loader:
            val_inputs_reduced = torch.mean(val_inputs, dim=-1)
            val_inputs = val_inputs_reduced.to(device)  # Send validation inputs to GPU if available

            # Forward pass
            val_outputs = autoencoder(val_inputs)
            val_loss = criterion(val_outputs, val_inputs)  # Compare reconstruction with original input
            total_val_loss += val_loss.item()

    # Calculate average validation loss
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Validation Loss: {avg_val_loss}")

    # Save the model if the validation loss has decreased
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(autoencoder.state_dict(), save_path)
        print(f"Model saved at epoch {epoch+1} with validation loss: {avg_val_loss}")

# Final save after training
torch.save(autoencoder.state_dict(), save_path)